# Analysis on the job ads
In this notebook I am analysing the stored job ads, with the goal of gathering interesting insights.

In [19]:
import json
import spacy
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from fastai.text import TextList

In [33]:
import nltk
nltk.download('wordnet')
nltk.download('punkt')
from nltk import stem

[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\Raffaele\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Raffaele\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping tokenizers\punkt.zip.


In [82]:
# Read JSON file
with open('data/job_ads_container_large.json') as data_file:
    job_ads_container = json.load(data_file)

In [83]:
len(job_ads_container)

167

## Text Lemmatization, Stemming and Vectorising with nlkt and sk-learn

In [84]:
wnl = stem.WordNetLemmatizer()
porter = stem.porter.PorterStemmer()

In [85]:
description_text = [job_ads_container[item]['Description'] for item in job_ads_container]

In [27]:
description_text[0]

' DESCRIPTION Alexa Shopping Spoken Language Understanding team is looking for a senior data scientist to join a recently established team in London. The team’s mission is to ‘put the customer in the loop’ to improve the shopping experience on Alexa. The team will be  building systems to collect and use explicit and implicit signals from customer behavior. As the lead scientist on the team you will drive the investigation to make this data actionable. You will have considerable scope to direct the research to find the maximum  impact. This could include identifying leading indicators of customer dissatisfaction, using customer provided signals to improve existing Alexa language understanding models or operationalizing entirely new models to improve the experience for our customers.This is a blue-sky role that gives you a chance to roll up your sleeves and dive into big data sets in order to build simulations and experimentation systems at scale, build optimization algorithms and levera

In [86]:
description_text_words = [nltk.word_tokenize(text) for text in description_text]

In [87]:
# splitting each text in words to perform lemmatization and stemming
description_text_lem = [[wnl.lemmatize(word) for word in text] for text in description_text_words]
description_text_stem = [[porter.stem(word) for word in text] for text in description_text_words]

In [88]:
# Re-joining the lists into a text
separator = ' '
descriptions_lem = [separator.join(text) for text in description_text_lem]
descriptions_stem = [separator.join(text) for text in description_text_stem]

In [63]:
descriptions_stem[0]

'descript alexa shop spoken languag understand team is look for a senior data scientist to join a recent establish team in london . the team ’ s mission is to ‘ put the custom in the loop ’ to improv the shop experi on alexa . the team will be build system to collect and use explicit and implicit signal from custom behavior . As the lead scientist on the team you will drive the investig to make thi data action . you will have consider scope to direct the research to find the maximum impact . thi could includ identifi lead indic of custom dissatisfact , use custom provid signal to improv exist alexa languag understand model or operation entir new model to improv the experi for our customers.thi is a blue-ski role that give you a chanc to roll up your sleev and dive into big data set in order to build simul and experiment system at scale , build optim algorithm and leverag cutting-edg technolog across amazon . thi is an opportun to think big about how to interpret the behavior of our cus

In [89]:
vectorizer = TfidfVectorizer(input='content', stop_words='english')
dtm = vectorizer.fit_transform(descriptions_stem).toarray()
vocab = np.array(vectorizer.get_feature_names())
dtm.shape, len(vocab)

((167, 4188), 4188)

Let's take a look at a random word in my vocabulary

In [92]:
vocab[1200]

'double'

In [93]:
dtm

array([[0.      , 0.      , 0.      , 0.      , ..., 0.      , 0.      , 0.      , 0.      ],
       [0.      , 0.      , 0.      , 0.      , ..., 0.      , 0.      , 0.      , 0.      ],
       [0.      , 0.103926, 0.      , 0.      , ..., 0.      , 0.      , 0.      , 0.      ],
       [0.      , 0.      , 0.      , 0.      , ..., 0.      , 0.      , 0.      , 0.      ],
       ...,
       [0.      , 0.018092, 0.      , 0.      , ..., 0.      , 0.      , 0.      , 0.      ],
       [0.      , 0.      , 0.      , 0.      , ..., 0.      , 0.      , 0.      , 0.      ],
       [0.      , 0.034021, 0.      , 0.      , ..., 0.      , 0.      , 0.      , 0.      ],
       [0.      , 0.      , 0.      , 0.      , ..., 0.      , 0.      , 0.      , 0.      ]])

In [95]:
scores = {'Words': vocab,'Scores':sum(dtm)}
word_scores = pd.DataFrame(data = scores)
word_scores.sort_values(by = ['Scores'],inplace = True, ascending = False)

In [99]:
key_skills = ['python',
             'experi','junior','senior',
             'sql',
             'nlp',
             'marketing', 'customer','market',
              'estate',
              'product','research','analytics','analyt',
             'statist','model',
             'recommender',
             'financial','fintech','financ',   
             'opencv','vision',
             'social',
             'java','c++',
             'deep','neural',
              'machin',
              'aw'
             ]

In [100]:
word_scores[word_scores.Words.isin(key_skills)]

,Words,Scores
1418,experi,8.985291
2916,product,6.043142
2226,machin,5.801777
2391,model,5.027774
3001,python,5.021593
271,analyt,4.549113
2275,market,4.085800
3176,research,3.909942
422,aw,3.651333
3398,senior,3.489841


In [98]:
word_scores[0:50]

,Words,Scores
1004,data,19.017357
4144,work,10.069307
1418,experi,8.985291
3848,thi,8.012704
3330,scienc,7.941769
2118,learn,7.635991
1098,develop,7.447096
1304,engin,7.296850
3768,team,7.206345
3343,scientist,6.151852


### _Based on the analysis, what are the most recurring key words in data science job ads?_  

 - **Experience**
 - **Product, Market**
 - **Modelling, Statistics**
 - **Python**
 - **SQL**
 - **NLP**
 - **AWS**
 - **Senior** rather than **Junior**
 - **Research, Insight, Analytics**
 - **Deep Learning**
 - **FinTech**

Computer vision is less present, along with social media and java.

## Comparing the advertised salary and contract type to the presence of keywords in the job description

I am going to restrict the analysis to the words contained in _key_skills_, as I don't want highly recurring words, such as "data science" and "work", to skew my analysis. This is obviously a personal choice and a very valid alternative approach would be to consider the 50 (or more) most common words, and to extend the analysis to more job ads. Unfortunately the market now (i.e. July 2020) sucks, and there aren't that many advertised positions online. The scraping notebook may be extended to include other job boards, which might provide a more well-rounded picture of the job market. another thing that could be done is to compare the scraped job descriptions, to make sure that there are no repetitions, e.g. same job advertised by multiple recruitment agencies.

In [101]:
features_indexes = word_scores[word_scores.Words.isin(key_skills)].index

In [102]:
factors_matrix = dtm[:,features_indexes]

In [103]:
factors_matrix.shape

(167, 24)

## WordCloud

In [105]:
from wordcloud import WordCloud

text = ''
for desc in description_text:
    text+=desc

# Generate a word cloud image
wordcloud = WordCloud().generate(text)

# Display the generated image:
# the matplotlib way:


# lower max_font_size
wordcloud = WordCloud(max_font_size=40).generate(text)
plt.figure(figsize=(15, 15)) #
plt.imshow(wordcloud) #, interpolation="bilinear"
plt.axis("off")
plt.savefig('data/wordcloud.png')
plt.show()